In [1]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os
import re

In [2]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-yG5q6JxtLgMS1Twv0mwZKbIVmJw5pNKGL6uU57p5IfYYJUq7',
)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.10`.
You can enable strict version checking by passing strict_version_check=True.


In [3]:
def ingest_documents(client: H2OGPTE):
    collection_id = None
    name = 'Musicrecco'

    recent_collections = client.list_recent_collections(0, 1000)
    for c in recent_collections:
        if c.name == name and c.document_count:
            collection_id = c.id
            break

    # Create Collection
    if collection_id is None:
        collection_id = client.create_collection(
            name=name,
            description='Music Recommender',
        )

        # Upload file into collection
        song_data = client.upload('song.xlsx', open('../../data/song.xlsx', 'rb'))
        client.ingest_uploads(collection_id, [song_data])

        print(f"DONE: {collection_id}")
    return collection_id

In [4]:
collection_id = ingest_documents(client)
chat_session_id = client.create_chat_session(collection_id)

In [22]:
[x for x in dir(client) if x[:1] != "_"]

['INITIAL_WAIT_INTERVAL',
 'MAX_WAIT_INTERVAL',
 'TIMEOUT',
 'WAIT_BACKOFF_FACTOR',
 'answer_question',
 'cancel_job',
 'connect',
 'count_assets',
 'count_chat_sessions',
 'count_chat_sessions_for_collection',
 'count_collections',
 'count_documents',
 'count_documents_in_collection',
 'count_documents_owned_by_me',
 'count_prompt_templates',
 'count_question_reply_feedback',
 'create_chat_session',
 'create_chat_session_on_default_collection',
 'create_collection',
 'create_prompt_template',
 'delete_chat_messages',
 'delete_chat_sessions',
 'delete_collections',
 'delete_document_summaries',
 'delete_documents',
 'delete_documents_from_collection',
 'delete_prompt_templates',
 'delete_upload',
 'download_document',
 'encode_for_retrieval',
 'extract_data',
 'get_chat_session_prompt_template',
 'get_chat_session_questions',
 'get_chunks',
 'get_collection',
 'get_collection_for_chat_session',
 'get_collection_prompt_template',
 'get_collection_questions',
 'get_default_collection',
 

In [23]:
print([x["base_model"] for x in client.get_llms()])

['mistralai/Mixtral-8x7B-Instruct-v0.1', 'h2oai/h2ogpt-4096-llama2-70b-chat', 'h2oai/h2ogpt-4096-llama2-13b-chat', 'h2oai/h2ogpt-32k-codellama-34b-instruct', 'HuggingFaceH4/zephyr-7b-beta', 'NousResearch/Nous-Capybara-34B', 'claude-2.1', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k-0613', 'gpt-35-turbo-1106', 'gpt-4-1106-preview', 'gemini-pro', 'mistral-small-latest', 'mistral-large-latest', 'mistral-medium']


In [24]:
client.get_prompt_template()

PromptTemplate(is_default=True, id=None, name='Defaults', description='Default h2oGPTe prompt template', lang='en', system_prompt='You are h2oGPTe, an expert question-answering AI system created by H2O.ai that performs like GPT-4 by OpenAI.', pre_prompt_query='Pay attention and remember the information below. You will need to use only the given document context to answer the question or imperative at the end.', prompt_query='According to only the information in the document sources provided within the context above, ', hyde_no_rag_llm_prompt_extension='Keep the answer brief, and list the 5 most relevant key words at the end.', pre_prompt_summary='In order to write a concise single-paragraph or bulleted list summary, pay attention to the following text:', prompt_summary='Using only the text above, write a condensed and concise summary of key results (preferably as bullet points).', system_prompt_reflection='You are acting as a judge. You must be fair and impartial and pay attention to d

In [27]:
genres = 'rock,pop,country'

In [5]:
prompt = "As an AI chatbot, you recommend songs based on user queries. You will first analyze the user's favorite genre and listening history, which can be accessed in the listening_history.xlsx file using the provided user ID. Then, you will match these preferences with similar songs from the song.xlsx file. This file includes data on Spotify ID, Song name, Artist, Rank, Genre, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, and tempo. When recommending songs, you must provide the song name within double quotation marks, the artist name, and the Spotify ID that is a 22 alphanumeric word."

In [6]:
prompt_template_id = client.create_prompt_template(
    name="Song Recommendation Template",
    description="Template for recommending songs",
    system_prompt = prompt,
    pre_prompt_query="Before providing recommendations, please provide your favorite music genre and userid.",
    prompt_query="Reccommend me a max of 5 songs based on my favourite genre with spotify id that is not in my lsitening history."
)


In [7]:
client.set_collection_prompt_template(collection_id, prompt_template_id)

'356147d7-f116-439e-9aab-90fcf8726c67'

In [8]:
client.get_chat_session_prompt_template(chat_session_id)

PromptTemplate(is_default=False, id='f66d35a0-1049-4a46-8245-fea00415b200', name='Song Recommendation Template', description='Template for recommending songs', lang=None, system_prompt="As an AI chatbot, you recommend songs based on user queries. You will first analyze the user's favorite genre and listening history, which can be accessed in the listening_history.xlsx file using the provided user ID. Then, you will match these preferences with similar songs from the song.xlsx file. This file includes data on Spotify ID, Song name, Artist, Rank, Genre, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, and tempo. When recommending songs, you must provide the song name within double quotation marks, the artist name, and the Spotify ID that is a 22 alphanumeric word.", pre_prompt_query='Before providing recommendations, please provide your favorite music genre and userid.', prompt_query='Reccommend me a max of 5 songs based on my favourite genr

In [11]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message="Some dance songs",
        rag_config={"rag_type": "rag"},
    ).content

    print(answer)

Based on your preference for dance music, here are five song recommendations with their Spotify IDs that are not in your listening history:

1. "Get Lucky" by Daft Punk (Spotify ID: 6rqhFgbbKwnb9MLmUQDhBI)
2. "Don't Stop the Music" by Rihanna (Spotify ID: 4cO5pRvC5z6UQiYOuTqo4C)
3. "One More Time" by Daft Punk (Spotify ID: 6rqhFgbbKwnb9MLmUQDhBI)
4. "Clarity" by Zedd ft. Foxes (Spotify ID: 5vIuDIiXz2sRoQ0AXq84aI)
5. "Titanium" by David Guetta ft. Sia (Spotify ID: 6rqhFgbbKwnb9MLmUQDhBI)

These songs have high danceability scores and are popular choices in the dance music genre. Enjoy listening!
